In [1]:

!pip install pathway --quiet


import pandas as pd
import numpy as np
from datetime import datetime
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
output_notebook()

import pathway as pw


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# 📥 Load and preprocess dataset
df = pd.read_csv('dataset.csv')

# Combine date and time into a single timestamp column
df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format="%d-%m-%Y %H:%M:%S")
df.drop(['LastUpdatedDate', 'LastUpdatedTime'], axis=1, inplace=True)
df = df.sort_values('timestamp').reset_index(drop=True)
df['occupancy_rate'] = df['Occupancy'] / df['Capacity']
df['is_special_day'] = df['IsSpecialDay'].map({'Yes': 1, 'No': 0})
vehicle_map = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
df['vehicle_weight'] = df['VehicleType'].map(vehicle_map)


df.head()
df[["timestamp", "Occupancy", "Capacity", "SystemCodeNumber"]].to_csv("parking_stream.csv", index=False)

In [4]:
# 🔧 Model 1: Baseline Linear Pricing
def baseline_price(prev_price, occupancy, capacity, alpha=0.1):
    """
    Increases price linearly with occupancy.
    """
    return prev_price + alpha * (occupancy / capacity)


In [5]:
# 🔧 Model 2: Demand-Based Pricing

# Define demand function
def compute_demand(row, weights):
    return (
        weights['occupancy'] * row['occupancy_rate'] +
        weights['queue'] * row['QueueLength'] -
        weights['traffic'] * row['TrafficConditionNearby'] +
        weights['special_day'] * row['is_special_day'] +
        weights['vehicle'] * row['vehicle_weight']
    )

# Normalize and apply pricing
def demand_based_price(base_price, demand, lambda_=0.5):
    norm_demand = (demand - demand.min()) / (demand.max() - demand.min())
    price = base_price * (1 + lambda_ * norm_demand)
    return np.clip(price, 0.5 * base_price, 2 * base_price)


In [6]:
# 🔧 Model 3: Competitive Pricing (Optional)
from geopy.distance import geodesic

def haversine_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).meters

def adjust_for_competition(current_price, nearby_prices, is_full):
    avg_nearby = np.mean(nearby_prices)
    if is_full and current_price > avg_nearby:
        return avg_nearby - 1  # Suggest rerouting
    elif current_price < avg_nearby:
        return current_price + 1  # Slight increase
    return current_price


In [7]:
import pathway as pw
# from datetime import datetime # This import is not needed for the schema definition

class ParkingInputSchema(pw.Schema):
    timestamp: str
    ID: str
    SystemCodeNumber: str
    Capacity: int
    Latitude: float
    Longitude: float
    Occupancy: int
    VehicleType: str
    TrafficConditionNearby: str
    QueueLength: int
    IsSpecialDay: str

In [8]:
input_table = pw.io.csv.read(
    "dataset.csv",
    schema=ParkingInputSchema,
    mode="streaming",  # simulate real-time
    autocommit_duration_ms=1000  # 1-second intervals
)


In [9]:
@pw.udf
def compute_price(row):
    occupancy_rate = row.Occupancy / row.Capacity
    is_special_day = 1 if row.IsSpecialDay == "Yes" else 0
    vehicle_weight = {"car": 1.0, "bike": 0.5, "truck": 1.5}.get(row.VehicleType, 1.0)

    demand = (
        0.4 * occupancy_rate +
        0.3 * row.QueueLength -
        0.2 * row.TrafficConditionNearby +
        0.1 * is_special_day +
        0.2 * vehicle_weight
    )

    norm_demand = min(max((demand - 0.5) / 1.5, 0), 1)
    price = 10 * (1 + 0.5 * norm_demand)
    return round(min(max(price, 5), 20), 2)


In [10]:
output_table = input_table.select(
    timestamp=input_table.timestamp,
    lot_id=input_table.SystemCodeNumber,
    price=compute_price(
        input_table.Occupancy,
        input_table.Capacity,
        input_table.QueueLength,
        input_table.TrafficConditionNearby,
        input_table.IsSpecialDay,
        input_table.VehicleType
    )
)


pw.run()

Output()

In [11]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from IPython.display import clear_output
import time

output_notebook()

# Initialize data source
source = ColumnDataSource(data=dict(time=[], price=[]))

# Create figure
p = figure(title="Simulated Real-Time Parking Price",
           x_axis_type='datetime', width=800, height=400)
p.line('time', 'price', source=source, line_width=2, color='navy')
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Price (₹)'


In [19]:
# Load simulated stream
stream_df = pd.read_csv("parking_stream.csv")
stream_df['timestamp'] = pd.to_datetime(stream_df['timestamp'])

# Simulate pricing logic
def simulate_price(occupancy, capacity):
    rate = occupancy / capacity
    demand = 0.4 * rate + 0.3 * np.random.randint(0, 5) - 0.2 * np.random.random()
    norm_demand = min(max((demand - 0.5) / 1.5, 0), 1)
    return round(10 * (1 + 0.5 * norm_demand), 2)

# Simulate real-time updates
from bokeh.io import push_notebook

handle = show(p, notebook_handle=True)  # Show once and get handle


from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category10
from bokeh.io import push_notebook
import time

output_notebook()

# Get unique parking lots
lot_ids = stream_df['SystemCodeNumber'].unique()
colors = Category10[10] * 2  # Enough colors for 14 lots

# Create a data source for each lot
sources = {
    lot: ColumnDataSource(data=dict(
        time=[], baseline=[], demand=[], model3=[], competitor=[]
    ))
    for lot in lot_ids
}


# Create figure
p = figure(title="Real-Time Pricing per Parking Lot", x_axis_type='datetime', width=900, height=500)
for i, lot in enumerate(lot_ids):
   p.line('time', 'baseline', source=sources[lot], color=colors[i], line_dash='dotdash', legend_label=f"{lot} - Model 1")
   p.line('time', 'demand', source=sources[lot], color=colors[i], line_width=2, legend_label=f"{lot} - Model 2")
   p.line('time', 'model3', source=sources[lot], color=colors[i], line_dash='dotted', legend_label=f"{lot} - Model 3")
   p.line('time', 'competitor', source=sources[lot], color=colors[i], line_dash='dashed', legend_label=f"{lot} - Competitor")


p.legend.click_policy = "hide"
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Price (₹)'

handle = show(p, notebook_handle=True)

for i, row in stream_df.iterrows():
    lot = row['SystemCodeNumber']
    timestamp = pd.to_datetime(row['timestamp'])
    # Model 1: Baseline
    baseline = 10 + 0.1 * (row['Occupancy'] / row['Capacity'])

    # Model 2: Demand-based
    demand_price = simulate_price(row['Occupancy'], row['Capacity'])

    # Model 3: Competitive
    competitor_price = round(demand_price + np.random.uniform(-2, 2), 2)
    model3_price = max(competitor_price - 0.5, 5)

    new_data = dict(
        time=[timestamp],
        baseline=[baseline],
        demand=[demand_price],
        model3=[model3_price],
        competitor=[competitor_price]
    )

    sources[lot].stream(new_data, rollover=100)

    push_notebook(handle=handle)
    time.sleep(0.2)

KeyboardInterrupt: 

In [14]:
%%capture --no-display
pw.run()


Output()